In [1]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


In [2]:
# read in known applicant files, dedupe and try to merge with applicants file
from collections import Counter
import difflib
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os

from data_cleaning_functions import (trans_remov_punc, standardize_whitespace, remove_punc, remove_suffix_from_last_name,
                                     clean_names, has_award, has_suffix, get_suffix, replace_last_name, 
                                     is_year_range, str_sim, clean_med_school, clean_std_college_name, long_form_date, 
                                    correct_mispellings, is_foreign_med_school)

from dev import (
    APP_DATA_DIR, SUM_STAT_DIR, ATT_DATA_DIR, CARD_DATA_DIR, CORRECTIONS_DIR, AWARDS_KEYWORDS, NAME_COLS, RAW_NAME_COLS, 
    RAW_CARD_ID, RAW_INDEX_IDS, PERSON_APPLICATION_ID, PERSON_ID, NIH_ID, FEMALE_FIRST_NAMES, FEMALE_MIDDLE_NAMES, 
    PICKLE_DIR)
from merging_functions import *

OUTPUT_CSV = False 

PERSONAL_INFO = [
    'clean_first_name', 'clean_last_name', 'clean_middle_name',
    'date_of_birth', 'medical_school', 'clean_college_trans']


time: 295 ms


In [3]:
# id column that links back to raw applicant data file
RAW_CARD_ID = 'raw_uuid'

# column where the raw id information is stored
RAW_INDEX_IDS = 'raw_card_ids'

# try to get one id per unique applicant in the dataset
PERSON_ID = 'person_uuid'
# id per deduped application-person - if someone applied multiple times, they will have multiple ids
PERSON_APPLICATION_ID = 'person_app_uuid' 
NIH_ID = 'dno'

APPLICANT_SUFFIX = '_ap'
ATTENDEE_SUFFIX = '_at'

%load_ext autoreload
%autoreload 2

%reload_ext autoreload

time: 38.5 ms


In [4]:
apps_filename = 'index_cards_deduped_fuzzy.csv'
# apps_filename = 'person_application_date_wide.csv'

NIH_filename = 'unique_attendees.csv'
# NIH_filename = 'NIH_attendee_deduped_raw.csv'

time: 12.2 ms


In [5]:
NAME_COLS = ['clean_middle_name', 'clean_last_name', 'clean_first_name']

MED_TRAINING_COLS = ['res_dates', 'intern_dates', 'residency_hospital', 'internship_hospital', 'medical_school', 'residency']


time: 11.9 ms


In [6]:
# import NIH raw data set
NIH_raw = pd.read_csv(os.path.join(ATT_DATA_DIR, NIH_filename)).drop_duplicates('dno')

time: 42.2 ms


In [7]:
sorted(NIH_raw.columns)

['Unnamed: 0',
 'application_year_max',
 'application_year_min',
 'citizenship',
 'clean_first_initial',
 'clean_first_name',
 'clean_last_name',
 'clean_medical_school',
 'clean_middle_initial',
 'clean_middle_name',
 'clean_suffix',
 'count_missing',
 'data_source',
 'dno',
 'dno_medical_school',
 'dob',
 'dup_flag',
 'duplicate_dno',
 'eod_year',
 'eod_year_diff',
 'generation',
 'institute',
 'intern_dte',
 'intern_hos',
 'lab_brch',
 'med_school',
 'medical_school',
 'program',
 'res_dtes',
 'res_hosp',
 'residency',
 'source',
 'ssn',
 'supervisor',
 'unknown',
 'year_grad']

time: 22 ms


In [8]:
# drop from the data set all people with eod years > 1980
# NIH = NIH_raw.loc[NIH_raw.eod_year<1980, :] 
NIH = NIH_raw
del NIH['medical_school']

time: 14.5 ms


In [9]:
# import cleaned, deduped applicant data set in wide form (multiple app dates as columns)
apps = pd.read_csv(os.path.join(APP_DATA_DIR, apps_filename))


time: 64 ms


In [10]:
apps.loc[:, 'res_dates'] = apps['residency_year(s)'].apply(long_form_date)

apps.loc[:, 'intern_dates'] = apps['internship_year(s)'].apply(long_form_date)

"Open"
Str. Medicine
Medicine
time: 56.6 ms


In [11]:
apps2 = apps.rename(
    columns={'residency_type': 'residency', 'internship_hospital_1': 'internship_hospital'})

time: 21.4 ms


In [12]:
NIH = NIH.rename(columns={'res_dtes':'res_dates', 'intern_dte':'intern_dates', 'intern_hos': 'internship_hospital', 
                         'res_hosp':'residency_hospital', 'clean_medical_school': 'medical_school'})
# sorted(NIH.columns)

time: 16.6 ms


In [13]:
NIH.loc[NIH.clean_last_name=='CHISARI', 'dno']

1013    623.0
Name: dno, dtype: float64

time: 22.6 ms


In [14]:
string_med_cols = ['medical_school', 'residency_hospital', 'internship_hospital', 'residency']

# apply string cleaning function to each of the string medical info columns
apps2.loc[:, string_med_cols] = apps2[string_med_cols].applymap(clean_names)

time: 184 ms


In [15]:
apps2.medical_school.dropna().unique()

array(['HARVARD MEDICAL SCHOOL', 'UNIVERSITY OF MIAMI SCHOOL OF MEDICINE',
       'NEW YORK UNIVERSITY COLLEGE OF MEDICINE',
       'UNIVERSITY OF LOUISVILLE SCHOOL OF MEDICINE',
       'USC KECK SCHOOL OF MEDICINE',
       'UNIVERSITY OF SOUTH CAROLINA SCHOOL OF MEDICINE',
       'GEORGETOWN UNIVERSITY SCHOOL OF MEDICINE',
       'ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA UNIVERSITY',
       'COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND SURGEONS',
       'LOUISIANA STATE UNIVERSITY SCHOOL OF MEDICINE IN NEW ORLEANS',
       'UCSF SCHOOL OF MEDICINE', 'UNIVERSITY OF MICHIGAN MEDICAL SCHOOL',
       'UNIVERSITY OF KANSAS SCHOOL OF MEDICINE',
       'WASHINGTON UNIVERSITY SCHOOL OF MEDICINE',
       'UNIVERSITY OF TEXAS HEALTH SCIENCES CENTER AT HOUSTON',
       'VANDERBILT UNIVERSITY SCHOOL OF MEDICINE',
       'TUFTS UNIVERSITY SCHOOL OF MEDICINE', 'DUKE MEDICAL SCHOOL',
       'UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE',
       'UMDNJ NEW JERSEY MEDICAL SCHOOL',
       'UNIVERSI

time: 19.7 ms


In [16]:
sorted(apps2.columns)

['address',
 'age',
 'application_date',
 'application_year',
 'application_year_1',
 'application_year_2',
 'application_year_3',
 'associate_program_entered',
 'bob',
 'ca',
 'cc',
 'citizenship',
 'city',
 'clean_college_trans',
 'clean_first_initial',
 'clean_first_name',
 'clean_last_name',
 'clean_middle_initial',
 'clean_middle_name',
 'clean_suffix',
 'clinical',
 'cord',
 'date_of_birth',
 'dbs',
 'fifth',
 'first_name',
 'honor_societies_first',
 'honor_societies_fourth',
 'honor_societies_second',
 'honor_societies_third',
 'ic',
 'intern_dates',
 'internship_hospital',
 'internship_year(s)',
 'is_female',
 'last_name',
 'medical_school',
 'medschool_year_grad',
 'middle_name',
 'nci',
 'nei',
 'nhi',
 'nhli',
 'niaid',
 'niamd',
 'niamdd',
 'nichd',
 'nichhd',
 'nidr',
 'niehs',
 'nigms',
 'nimh',
 'nindb',
 'ninds',
 'oir',
 'original_medical_school',
 'other',
 'person_uuid',
 'pharm_ra',
 'pi',
 'ra',
 'raw_uuid',
 'raw_uuid_1',
 'raw_uuid_2',
 'raw_uuid_3',
 'raw_uuid_4

time: 19.6 ms


In [17]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

apps2.loc[:, 'clean_last_name'] = apps2.clean_last_name.apply(correct_name_mispellings_fnc)
apps2.loc[:, 'clean_first_name'] = apps2.clean_first_name.apply(correct_name_mispellings_fnc)

time: 73.5 ms


In [18]:
# there are a bunch of duplicates in apps, where application year is the same, but first name is missing
name_dups = apps2.loc[
    apps2.duplicated(
        ['clean_last_name', 'medical_school', 'application_year'], keep=False), NAME_COLS+MED_TRAINING_COLS]

time: 17.8 ms


In [19]:
# name_dups.sort_values(['clean_last_name'])

time: 11.4 ms


In [20]:
apps3 = apps2.sort_values(['clean_last_name', 'clean_first_name']).drop_duplicates(
    ['clean_last_name', 'medical_school', 'application_year'])

time: 28.8 ms


In [21]:
# there are also some duplicates on middle and last name, but first is missing on one of the dups, 
# so we need to drop these
dups_mask = apps3.duplicated(['clean_last_name', 'clean_middle_name', 'application_year'], keep=False)

time: 15.2 ms


In [22]:
apps4 = apps3.loc[((~dups_mask) | ((dups_mask) & (~pd.isnull(apps3.clean_first_name)))), :]

time: 32.1 ms


In [23]:
print apps4.shape
print apps3.shape

(4123, 84)
(4130, 84)
time: 13 ms


In [24]:
apps4.loc[apps4.clean_last_name=='HAAKENSTAD', NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name
59,OTTO,HAAKENSTAD,ALAN


time: 22.9 ms


In [25]:
apps3.loc[apps3.clean_last_name=='HAAKENSTAD', NAME_COLS+['clean_medical_school', 'address', 'city']]

,clean_middle_name,clean_last_name,clean_first_name,clean_medical_school,address,city
59,OTTO,HAAKENSTAD,ALAN,NaN,1718 7th Street South,South Fargo


time: 26.6 ms


In [26]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        df.loc[df[selection_type]==selection_value, t] = v

time: 13.3 ms


In [27]:
%%capture
change_names(
    apps4, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(
    apps4, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    apps4, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    apps4, 'clean_last_name', 'EILER', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    apps4, 'clean_last_name', 'FALCHUK', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    apps4, 'clean_last_name', 'BOYD', ['clean_first_name', 'clean_middle_name'], ['MICHAEL', 'RAY'])
change_names(
    apps4, 'clean_last_name', 'CHAPMAN', 
    ['clean_first_name', 'clean_middle_name', 'medical_school'], ['STANLEY', 'WILLETS', 'ROCHESTER'])
change_names(apps4, 'clean_last_name', 'DANFORTH', ['clean_first_name'], ['DAVID'])
change_names(apps4, 'clean_last_name', 'HUNT', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'D'])
change_names(apps4, 'clean_last_name', 'KARK', ['clean_first_name', 'clean_middle_name'], ['ROBERT', 'ADRIAN'])
change_names(apps4, 'clean_last_name', 'KEBABIAN', ['clean_first_name', 'clean_middle_name'], ['JOHN', 'WILLIS'])
change_names(apps4, 'clean_last_name', 'KNOPF', ['clean_first_name', 'clean_middle_name'], ['HARRY', 'LOUIS'])
change_names(apps4, 'clean_last_name', 'KROLIKOWSKI', ['clean_first_name', 'clean_middle_name'], ['FRANCIS', 'JOHN'])
change_names(apps4, 'clean_last_name', 'KASTL', ['clean_first_name', 'clean_middle_name'], ['DAVID', 'GENE'])
change_names(apps4, 'clean_first_name', 'JAN', ['clean_last_name'], ['KNOWLER'])
change_names(apps4, 'clean_last_name', 'KLAVEMAN', ['clean_last_name'], ['KLAEVEMAN'])
change_names(apps4, 'clean_last_name', 'MATHEW', ['clean_last_name'], ['MATTHEW'])

apps4.loc[apps4.clean_last_name=='CHESEBRO', ['clean_first_name']] = 'BRUCE'
apps4.loc[apps4.clean_last_name=='CHESEBRO', ['clean_middle_name']] = 'WILCOX'
apps4.loc[(apps4.clean_last_name=='HEALY') & (apps4.medical_school=='USC KECK'), ['clean_first_name']] = 'MARK'
apps4.loc[(apps4.clean_last_name=='HEALY') & (apps4.medical_school=='USC KECK'), ['clean_middle_name']] = 'H'

time: 757 ms


In [28]:
apps4.loc[apps4.clean_last_name=='LENN', 'clean_first_name'] = 'NICHOLAS'
apps4.loc[apps4.clean_last_name=='BRESLOW', 'clean_first_name'] = 'JAN'
apps4.loc[apps4.clean_last_name=='BRESLOW', 'clean_middle_name'] = 'LESLIE'

time: 90.3 ms


In [29]:
apps4.loc[(apps4.clean_last_name=='NADLER') & (pd.isnull(apps4.clean_first_name)), 'clean_first_name'] = 'LEE'
apps4.loc[(apps4.clean_last_name=='NADLER') & (apps4.clean_first_name=='LEE'), 'clean_middle_name'] = 'MARSHALL'
apps4.loc[(pd.isnull(apps4.clean_first_name)) & (apps4.clean_last_name=='ROSEN'), 'clean_first_name'] = 'HENRY'

time: 88.4 ms


In [30]:
apps4.loc[(apps4.clean_last_name=='NEELON'), 'clean_first_name'] = 'FRANCIS'
apps4.loc[(apps4.clean_last_name=='NEELON') , 'clean_middle_name'] = 'ALBERT'

time: 65.4 ms


In [31]:
apps4.loc[(apps4.clean_last_name=='NICHOLAS') , 'clean_first_name']


1880    JOHN
Name: clean_first_name, dtype: object

time: 17.4 ms


In [32]:
NIH.loc[(NIH.clean_last_name=='ALEXANDER') & (NIH.clean_first_name=='JOHN'), 'clean_middle_name'] = 'CHARLES'

time: 16.7 ms


In [33]:
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_middle_name'] = 'ARTHUR'
NIH.loc[(NIH.clean_last_name=='LEBOWITZ') & (NIH.clean_first_name=='EDWARD ARTHUR'),'clean_first_name'] = 'EDWARD'
NIH.loc[(
        NIH.clean_last_name=='LEBOWITZ') & (
            NIH.clean_first_name=='EDWARD'), 'medical_school'] = 'ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA'

time: 22.4 ms


In [34]:
NIH.loc[(NIH.clean_last_name=='CHISARI'), NAME_COLS+['dno']]

,clean_middle_name,clean_last_name,clean_first_name,dno
1013,VINCENT,CHISARI,FRANCIS,623.0


time: 21 ms


In [36]:
# read in manual matches
man = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_dno_matches.xlsx'), index=False).rename(columns={'medical_school': 'dno_medical_school'})
man['medical_school'] = man.dno_medical_school.apply(clean_med_school)

COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND SURGEONS
COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND SURGEONS
HARVARD MEDICAL SCHOOL
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
RUTGERS NEW JERSEY MEDICAL SCHOOL
HARVARD MEDICAL SCHOOL
HARVARD MEDICAL SCHOOL
YALE SCHOOL OF MEDICINE
DUKE UNIVERSITY SCHOOL OF MEDICINE
ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
STATE UNIVERSITY OF NEW YORK DOWNSTATE MEDICAL CENTER COLLEGE OF MEDICINE
DUKE UNIVERSITY SCHOOL OF MEDICINE
UNIVERSITY OF ALABAMA
COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND SURGEONS
UNIVERSITY OF OKLAHOMA COLLEGE OF MEDICINE
HARVARD MEDICAL SCHOOL
UNIVERSITY OF OKLAHOMA COLLEGE OF MEDICINE
ALBERT EINSTEIN COLLEGE OF MEDICINE
HARVARD MEDICAL SCHOOL
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND SURGEONS
UNIVERSITY OF OKLAHOMA COLLEGE OF MEDICINE
HARVARD MEDICAL SCHOOL
VANDERBILT UNI

In [37]:
apps4.loc[apps4.clean_last_name=='NEELON', NAME_COLS]

,clean_middle_name,clean_last_name,clean_first_name
979,ALBERT,NEELON,FRANCIS


time: 28.2 ms


In [38]:
t = pd.merge(left=apps4, right=man, on=['clean_last_name', 'clean_middle_name'], how='inner', suffixes=['_x', '_y'])
t.shape

t['sim'] = t[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

t['clean_first_name'] = t['clean_first_name_y']

t_1 = t.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')

# t[NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]
t_1.loc[t_1.duplicated('dno', keep=False), NAME_COLS+['dno', 'medical_school_x', 'medical_school_y', 'sim']]

,clean_middle_name,clean_last_name,clean_first_name,dno,medical_school_x,medical_school_y,sim


time: 56.2 ms


In [39]:
um = man[~man.dno.isin(t_1.dno)]

um.shape

(9, 8)

time: 19.1 ms


In [42]:
um

,dno,clean_first_name,clean_middle_name,clean_last_name,application_year_min,application_year_max,dno_medical_school,medical_school
1,224,MYLES,MICHAEL,BEHRENS,1962,1962,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...
9,427,JAN,LESLIE,BRESLOW,1968,1968,HARVARD MEDICAL SCHOOL,HARVARD MEDICAL SCHOOL
15,645,RICHARD,AUGUST,CLARK,1971,1971,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND...,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND...
26,1163,BERNARD,LEONARD,FRANKEL,1967,1967,ALBERT EINSTEIN COLLEGE OF MEDICINE,ALBERT EINSTEIN COLLEGE OF MEDICINE
31,1393,SHELDON,HERSH,GOTTLIEB,1971,1971,MEDICAL COLLEGE OF WISCONSIN,MEDICAL COLLEGE OF WISCONSIN
48,2210,ROBERT,ISAAC,LEVY,1961,1961,YALE SCHOOL OF MEDICINE,YALE SCHOOL OF MEDICINE
67,2766,JAN,MARK,ORENSTEIN,1971,1971,STATE UNIVERSITY OF NEW YORK DOWNSTATE MEDICAL...,STATE UNIVERSITY OF NEW YORK DOWNSTATE MEDICAL...
74,3313,ANDREW,B,SCHWARTZ,1969,1969,YALE,YALE UNIVERSITY SCHOOL OF MEDICINE
99,4031,MITCHEL,LEONE,WOLF,1968,1968,ALBERT EINSTEIN COLLEGE OF MEDICINE,ALBERT EINSTEIN COLLEGE OF MEDICINE


time: 34.3 ms


In [43]:
t2 = pd.merge(left=apps4, right=um, on=['clean_last_name', 'clean_first_name'], how='inner', suffixes=['_x', '_y'])
print t2.shape
print um.shape

t2['sim'] = t2[['medical_school_x', 'medical_school_y']].apply(get_name_str_sim, axis=1)

t2['clean_middle_name'] = t2['clean_middle_name_y']

t2_1 = t2.sort_values(['dno', 'sim'], ascending=False).drop_duplicates(['dno'], keep='first')
t2_1.shape

(8, 90)
(9, 8)


(7, 92)

time: 49.1 ms


In [44]:
# t3 = t_1[NAME_COLS+['dno', PERSON_ID]]
t3 = pd.concat([t2_1[NAME_COLS+['dno', PERSON_ID]], t_1[NAME_COLS+['dno', PERSON_ID]]], axis=0)

time: 29.6 ms


In [45]:
man_dno = pd.merge(left=apps4, right=t3, on=PERSON_ID, how='left', suffixes=['_x', '_y'])
print man_dno.shape
print t3.shape
print man.shape

(4124, 88)
(101, 5)
(104, 8)
time: 45.2 ms


In [46]:
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno = man_dno.rename(columns={'clean_last_name_x': 'clean_last_name', 'clean_first_name_x': 'clean_first_name', 
                       'clean_middle_name_x': 'clean_middle_name'})
mask = ~pd.isnull(man_dno.clean_last_name_y)
man_dno.loc[mask, 'clean_last_name'] = man_dno.loc[mask, 'clean_last_name_y']
mask = ~pd.isnull(man_dno.clean_first_name_y)
man_dno.loc[mask, 'clean_first_name'] = man_dno.loc[mask, 'clean_first_name_y']
mask = ~pd.isnull(man_dno.clean_middle_name_y)
man_dno.loc[mask, 'clean_middle_name'] = man_dno.loc[mask, 'clean_middle_name_y']

time: 46.8 ms


In [47]:
apps5 = man_dno.loc[pd.isnull(man_dno['dno']), :].drop(['dno', 'clean_middle_name_y', 'clean_first_name_y', 'clean_last_name_y'], axis=1) 

time: 30.9 ms


In [48]:
# remove females from data set
female_mask = (NIH.clean_first_name.isin(FEMALE_FIRST_NAMES)| NIH.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))  
NIH = NIH.loc[~female_mask, :]

female_mask = (apps5.clean_first_name.isin(FEMALE_FIRST_NAMES)| apps5.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))  
apps5 = apps5.loc[~female_mask, :]

time: 32.9 ms


In [49]:
def get_first_letter(str_var):
    if pd.isnull(str_var) or str_var=='':
        return np.nan
    return str_var[0]

time: 27.6 ms


In [50]:
NIH['clean_first_initial'] = NIH.clean_first_name.apply(get_first_letter)
NIH['clean_middle_initial'] = NIH.clean_middle_name.apply(get_first_letter)

time: 50.5 ms


In [51]:
print NIH.shape
print  apps5.shape

(3859, 35)
(4023, 84)
time: 18.4 ms


In [52]:
# After cleaning apps2 to match cleaning in Clean NIH Applicant notebook, we try to start merging
sims_cols = ['medical_school_sim', 'clean_middle_name_sim', 'clean_first_name_sim']

time: 29.1 ms


In [53]:
NIH['fuzzy_merge_col'] = NIH[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
apps5['fuzzy_merge_col'] = apps5[
    ['clean_first_name', 'clean_middle_name', 'clean_last_name']].apply(create_str_merge, axis=1)
match1 = df_get_closest_matches(apps5, NIH, 'fuzzy_merge_col', suffixes=['_x', '_y']) 
print match1.shape

(3880, 119)
time: 5min 29s


In [54]:
# add last name counter to each
app_counter = Counter(apps4.clean_last_name.values)
NIH_counter = Counter(NIH.clean_last_name.values)
match1['last_name_counts_x'] = match1.clean_last_name_x.apply(lambda x: app_counter[x])
match1['last_name_counts_y'] = match1.clean_last_name_y.apply(lambda x: NIH_counter[x])


time: 27.8 ms


In [55]:
def check_match(row):
    # address and application year match
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if row['clean_last_name_sim'] < 60 or app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 80:
        return 1
    if row['last_name_counts_x'] < 2 and row['last_name_counts_y'] < 2:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 5):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 50:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 5) and row['medical_school_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 5) and row['clean_first_name_sim'] > 80:
        return 1
    return 0


feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'clean_last_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

match2 = add_similarity_features(match1, feature_dict, check_match, suffixes=['_x', '_y'])

time: 3.02 s


In [56]:
# match2.loc[(match2.is_match==2), sims_cols+['is_match', 'clean_first_name_x', 'clean_first_name_y',
#                         'clean_last_name_x', 'clean_last_name_y','medical_school_x', 'medical_school_y', PERSON_ID, NIH_ID]]

time: 11.2 ms


In [57]:
# select out people who match and make sure each person id and dno only 1x in data set
match3 = match2[match2.is_match==1].sort_values(['clean_last_name_x']+sims_cols, ascending=False).dropna(
    axis=0, subset=[RAW_CARD_ID])
print match3.shape

(2695, 126)
time: 76.2 ms


In [58]:
# match2[match2.clean_last_name_x=='ADLER']
NIH[NIH.clean_last_name=='ADLER']

,Unnamed: 0,application_year_max,application_year_min,citizenship,clean_first_initial,clean_first_name,clean_last_name,medical_school,clean_middle_initial,clean_middle_name,...,program,res_dates,residency_hospital,residency,source,ssn,supervisor,unknown,year_grad,fuzzy_merge_col
2947,0,NaN,NaN,NaN,R,RICHARD,ADLER,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,D,DAVID,...,CA,1970-1971,Johns Hopkins Hospital,Pediatrics,NIH EDUCATION,NaN,Whedon,NaN,1969.0,RICHARD DAVID ADLER
3059,2032,NaN,NaN,NaN,R,RICHARD,ADLER,VANDERBILT UNIVERSITY SCHOOL OF MEDICINE,NaN,NaN,...,CA,1960-1961,Johns Hopkins Hospital,Pediatrics,NIH EDUCATION,NaN,NaN,NaN,1959.0,RICHARD ADLER
3708,1849,NaN,NaN,NaN,S,STUART,ADLER,JOHNS HOPKINS UNIVERSITY SCHOOL OF MEDICINE,P,PHILLIP,...,NaN,NaN,NaN,NaN,NIH EDUCATION,NaN,NaN,NaN,1971.0,STUART PHILLIP ADLER


time: 76 ms


In [59]:
def filter_one_match_per_group(df, dedupe_col, sim_cols):
    # to merge cols should be a dict the names of the extra cols to merge in
    # values should be col names to rename
    # sim cols should be name of the columns to use as features
    # sim mask should be mask that accounts as actual mask
    # dedupe col is name of col to dedupe on

    def count_matches(id_list_arr):
        # for each id, make sure matched on 1x in data set
        # should be applied with rolling apply so takes in a dataframe and must return single value
        # unpack already matched ids from string
        current_id1 = id_list_arr[-1]
        other_matches = id_list_arr[:-1]
        is_dup = np.any(other_matches[:] == current_id1)
        if is_dup:
            return True
        return False

    # for each uuid, check for duplicates and choose best match based on sim cols
    # order of the sim cols should be with most important first
    dup_flag = '{}_duplicate'.format(dedupe_col)
    df[dup_flag] = 0
    df.loc[:, dup_flag] = df[
        dedupe_col].expanding(center=False, min_periods=0).apply(func=count_matches)

    df_matches = df[df['is_match'] == 1].sort_values([dedupe_col] + sim_cols, ascending=False)
    return df_matches.drop_duplicates([dedupe_col], keep='first')


time: 20.4 ms


In [60]:
match4 = filter_one_match_per_group(match3, 'raw_uuid', sims_cols)
print match4.shape

(2635, 127)
time: 143 ms


In [61]:
match5 = filter_one_match_per_group(match4, NIH_ID, sims_cols)
print match5.shape

(2625, 128)
time: 87.3 ms


In [62]:
# get nonmatched NIH people and not matched applicants 
nm_apps = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match5[RAW_CARD_ID].dropna().values)
nm_NIH = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match5[NIH_ID].dropna().values)

(1391, 85)
(1234, 36)
time: 19.6 ms


In [63]:
# do another round of matching just on last name
nm_match1 = df_get_closest_matches(nm_apps, nm_NIH, 'clean_last_name', suffixes=['_x', '_y']) 
print nm_match1.shape


(2022, 119)
time: 9.37 s


In [64]:
def check_match(row):
    app_eod_year_diff = abs(row['application_year'] - row['eod_year'])
    if app_eod_year_diff > 8:
        return 0
    
    # the first and middle name seem to be mixed up in index card data set
    mixed_sim1 =  get_name_str_sim(row[['clean_middle_name_x', 'clean_first_name_y']])
    mixed_sim2 =  get_name_str_sim(row[['clean_middle_name_y', 'clean_first_name_x']])
    
    max_name_sim = max(row['clean_first_name_sim'], row['medical_school_sim'], row['clean_middle_name_sim'])
    if max_name_sim < 60:
        return 0
    mix_sim = max(mixed_sim1, mixed_sim2)
    if pd.isnull(mix_sim):
        mix_sim = 0
    if (mix_sim > 90) and row['medical_school_sim'] > 60:
        return 1
    if (mix_sim > 90) and pd.isnull(row['medical_school_sim']) and (app_eod_year_diff < 6):
        return 1
    if not pd.isnull(row['clean_first_name_sim']) and row['clean_first_name_sim'] < 40:
        return 0
    # if matching application year and med schools match
    if (app_eod_year_diff < 6) and row['medical_school_sim'] > 80:
        return 1
    if not pd.isnull(row['medical_school_sim']) and row['medical_school_sim'] < 40:
        return 0
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_first_name_sim'] > 80:
        return 1
    # first and middle names match or first
    if (app_eod_year_diff < 6) and row['clean_middle_name_sim'] > 80:
        return 1
    return 0

time: 25.2 ms


In [65]:
nm_feature_dict = {
    'clean_first_name': get_name_str_sim,
    'clean_middle_name': get_name_str_sim,
    'medical_school': get_name_str_sim,
}

nm_match2 = add_similarity_features(nm_match1, nm_feature_dict, check_match, suffixes=['_x', '_y'])


time: 956 ms


In [66]:
nm_match3 = nm_match2.loc[(nm_match2.is_match==1) & (nm_match2.index!='MORTON'), :].reset_index(
    drop=False).rename(columns={'index': 'clean_last_name'})

time: 16 ms


In [67]:
# merge the manual people with dno with the dno data set and append to the matches list
man_dno2 = man_dno.loc[~pd.isnull(man_dno['dno']), :].drop(['clean_first_name_y', 'clean_middle_name_y',
                                                            'clean_last_name_y'], axis=1)
man_dno2.loc[:, 'dno'] = man_dno2.dno.astype(int)
NIH.loc[:, 'dno'] = NIH.dno.astype(int)

time: 18.7 ms


In [68]:
print man_dno2.shape
man_dno_merge = pd.merge(left=man_dno2, right=NIH, on='dno', how='inner')
print man_dno_merge.shape

(101, 85)
(95, 120)
time: 25.9 ms


In [69]:
man_dno_merge2 = man_dno_merge.rename(columns={'clean_first_name_x': 'clean_first_name2', 
                                               'clean_middle_name_x': 'clean_middle_name2',
                             'clean_last_name_x': 'clean_last_name2'})


time: 15.2 ms


In [70]:
fails = man_dno2.loc[~man_dno2.dno.isin(man_dno_merge.dno), :]
print fails.shape

(6, 85)
time: 14 ms


In [71]:
# append matches together
match6 = pd.concat([nm_match3, match5, man_dno_merge2], axis=0)

time: 49.9 ms


In [72]:
# get nonmatched NIH people and not matched applicants 
nm_apps2 = get_nonmatched(apps5, id_colname=RAW_CARD_ID, matched_ids=match6[RAW_CARD_ID].dropna().values)
nm_NIH2 = get_nonmatched(NIH, id_colname=NIH_ID, matched_ids=match6[NIH_ID].dropna().values)

(1321, 85)
(1093, 36)
time: 22.8 ms


In [73]:
nm_NIH3 = nm_NIH2.loc[
    (nm_NIH2.eod_year< 1976) & (
        nm_NIH2.eod_year>1963), NAME_COLS+['medical_school', 'eod_year']].sort_values('clean_last_name')
print nm_NIH3.shape

(51, 5)
time: 17.5 ms


In [74]:
apps_match = nm_apps2.loc[nm_apps2.clean_last_name.isin(nm_NIH3.clean_last_name.values)]
test_merge = pd.merge(left=nm_NIH3, right=apps_match, on='clean_last_name', how='inner').sort_values('clean_last_name')
test_merge = test_merge[sorted(test_merge.columns)]
if OUTPUT_CSV:
    test_merge.to_csv(os.path.join(CORRECTIONS_DIR, 'test_merge_missing_NIH.csv'), index=False)

time: 25.2 ms


In [75]:
# test_merge[['clean_first_name_x', 'clean_first_name_y', 'clean_middle_name_x', 'clean_middle_name_y']]

time: 11 ms


In [76]:
match6.loc[pd.isnull(match6.clean_last_name), 'clean_last_name'] =  match6.loc[
    pd.isnull(match6.clean_last_name), 'clean_last_name_x']
to_drop = [c for c in match6.columns if c.endswith('_sim') or '_counts' in c]
match7= match6.drop(to_drop+[
        'dup_flag', 'eod_year_diff', 'fuzzy_merge_col_x', 'fuzzy_merge_col_y', 'unknown',
        'Unnamed: 0', 'raw_uuid_duplicate', 'dno_duplicate', 'count_missing',
                'clean_last_name_x', 'clean_last_name_y', 'is_match'], axis=1)

time: 27.3 ms


In [77]:
# consolidate columns in match6
match7a = consolidate_merge_cols(match7, ['_x', '_y'], [])

['citizenship', 'clean_first_initial', 'clean_first_name', 'clean_middle_initial', 'clean_middle_name', 'clean_suffix', 'intern_dates', 'internship_hospital', 'medical_school', 'res_dates', 'residency_hospital', 'residency']
time: 14 s


In [78]:
mask = ~pd.isnull(match7a.clean_first_name2)
match7a.loc[mask, 'clean_first_name'] = match7a.loc[mask, 'clean_first_name2']

mask = ~pd.isnull(match7a.clean_last_name2)
match7a.loc[mask, 'clean_last_name'] = match7a.loc[mask, 'clean_last_name2']

mask = ~pd.isnull(match7a.clean_middle_name2)
match7a.loc[mask, 'clean_middle_name'] = match7a.loc[mask, 'clean_middle_name2']

time: 20 ms


In [79]:
match8 = pd.concat([
        man_dno[~pd.isnull(man_dno['dno'])], match7a, apps4.loc[
            ~apps4[PERSON_ID].isin(match7a[PERSON_ID].values),:]], axis=0).drop(['clean_first_name2', 
                                                                                'clean_middle_name2', 
                                                                                'clean_last_name2'], axis=1)
print sorted(match8.columns)
print match8.shape

['address', 'age', 'application_date', 'application_year', 'application_year_1', 'application_year_2', 'application_year_3', 'application_year_max', 'application_year_min', 'associate_program_entered', 'bob', 'ca', 'cc', 'citizenship', 'city', 'clean_college_trans', 'clean_first_initial', 'clean_first_name', 'clean_first_name_y', 'clean_last_name', 'clean_last_name_y', 'clean_middle_initial', 'clean_middle_name', 'clean_middle_name_y', 'clean_suffix', 'clinical', 'cord', 'data_source', 'date_of_birth', 'dbs', 'dno', 'dno_medical_school', 'dob', 'duplicate_dno', 'eod_year', 'fifth', 'first_name', 'fuzzy_merge_col', 'generation', 'honor_societies_first', 'honor_societies_fourth', 'honor_societies_second', 'honor_societies_third', 'ic', 'institute', 'intern_dates', 'internship_hospital', 'internship_year(s)', 'is_female', 'lab_brch', 'last_name', 'med_school', 'medical_school', 'medschool_year_grad', 'middle_name', 'nci', 'nei', 'nhi', 'nhli', 'niaid', 'niamd', 'niamdd', 'nichd', 'nichhd'

In [80]:
# replace date of birth with dob whenever date of birth missing and dob is not
match8.loc[
    (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'date_of_birth'] = match8.loc[
        (pd.isnull(match8['date_of_birth'])) & (~pd.isnull(match8['dob'])), 'dob']


c1 = 'date_of_birth'
c2 = 'dob'
match8.loc[(pd.isnull(match8[c1])) & (~pd.isnull(match8[c2])), [c1, c2]]

,date_of_birth,dob


time: 29.2 ms


In [81]:
match8.loc[(match8.clean_last_name=='KNOWLER') & (match8.clean_first_name=='JAN'), 'dno'] = 1922
match8.loc[
    (match8.clean_last_name== 'E ROSS HARVARD') & (match8.clean_first_name=='MICHAEL'), 'clean_last_name'] = 'ROSS'


time: 19.8 ms


In [82]:
match9 = match8.drop(['dob'], axis=1).reset_index(drop=True).sort_values(
    NAME_COLS+['dno']).drop_duplicates(NAME_COLS+['dno']).drop_duplicates(NAME_COLS+[PERSON_ID])
# print sorted(match9.columns)
print match8.shape
print match9.shape

(4236, 105)
(4130, 104)
time: 50.3 ms


In [85]:
# create a clean (standardized medical school column and remove people with international med schools)
match9['medical_school'] = match9['dno_medical_school']
mask = pd.isnull(match9.medical_school)
match9.loc[mask, 'medical_school'] = match9.loc[mask, 'original_medical_school']


match9['clean_medical_school'] = match9.medical_school.apply(clean_med_school)

OTHER
DUKE UNIVERSITY SCHOOL OF MEDICINE
UNIVERSITY OF CHICAGO PRITZKER SCHOOL OF MEDICINE
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
DUKE UNIVERSITY SCHOOL OF MEDICINE
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
OTHER
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
UNIVERSITY OF CHICAGO PRITZKER SCHOOL OF MEDICINE
STANFORD UNIVERSITY SCHOOL OF MEDICINE
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
HAHNEMANN UNIVERSITY SCHOOL OF MEDICINE
NEW YORK UNIVERSITY SCHOOL OF MEDICINE
UNIVERSITY OF CHICAGO PRITZKER SCHOOL OF MEDICINE
UNIVERSITY OF CHICAGO PRITZKER SCHOOL OF MEDICINE
UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND DENTISTRY
DUKE U

In [96]:
# add an international applicant flag
# also drop anyone missing first, middle names and med school
match9['is_foreign'] = 0
match9.loc[:, 'is_foreign'] = match9.clean_medical_school.apply(is_foreign_med_school)

time: 2.71 s


In [107]:
match9.loc[match9.med_school=='McGill University', :]

,address,age,application_date,application_year,application_year_1,application_year_2,application_year_3,application_year_max,application_year_min,associate_program_entered,...,teaching,undergrad_year_grad,undergraduate_school,withdrawal,year_accepted,year_grad,zip_code,clean_medical_school,IS_FOREIGN,is_foreign
2087,600 Main Street,NaN,1963-04-30,1963.0,NaN,NaN,NaN,NaN,NaN,CA,...,1.0,NaN,NaN,-9.0,1965.0,1963.0,NaN,OTHER,False,0.0
556,"3771 West 16th Avenue, Vancouver, B.C., Canada",NaN,1966-04-28,1966.0,NaN,NaN,NaN,NaN,NaN,CA,...,1.0,NaN,NaN,0.0,1968.0,1966.0,NaN,OTHER,False,0.0


time: 47.6 ms


In [106]:
match9.med_school.sort_values().dropna().unique()

array(['Albany', 'Albany Medical', 'Albany Medical Center',
       'Albany Medical College', 'Albany Medical School',
       'Albert Einstein', 'Albert Einstein Col. Medicine',
       'Albert Einstein College OF Medicine',
       'Albert Einstein College of Medicine',
       'Albert Einstein College of Medicine, N.Y.,N.Y.',
       'Albert Einstein College of Medicine.',
       'Albert Einstein School of Medicine',
       'Albert Einstein; Syracuse University',
       'Albert. Einstein College of Medicine', 'Baylor',
       'Baylor College of Medicine', 'Baylor Medical School',
       'Baylor University', 'Baylor University Col. of Med.',
       'Baylor University College of Medicine', 'Boston City Hospital',
       'Boston University', 'Boston University Medical School',
       'Boston University School of Medicine', 'Bowman Gray',
       'Bowman Gray Medical School; Duke University',
       'Bowman Gray School of Medicine',
       'Bowman Gray School of Medicine, Winston-Salem, N.C.',

time: 26.2 ms


In [97]:
match9.is_foreign.describe()

count    4130.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: is_foreign, dtype: float64

time: 17 ms


In [ ]:
match9['control_flag'] = 0
match9.loc[pd.isnull(match9.dno) & pd.isnull(match9.year_accepted), 'control_flag'] = 1

In [ ]:
match9.rename(columns={'res_dates': 'residency_dates', 'intern_dates': 'internship_dates', 
                      'clean_college_trans': 'clean_college'}, inplace=True)

IMPORTANT_COLS = [NIH_ID, PERSON_ID, 'application_year', 'eod_year', 'application_date', 'clean_first_name', 'clean_middle_name', 
                 'clean_last_name', 'control_flag', 'year_accepted', 'rejected', 'rejection_date', 'clean_college', 'medical_school',
                'residency_dates', 'internship_dates']

other_cols = sorted([i for i in match9.columns if i not in IMPORTANT_COLS])

# order columns so important ones are 
match10 = match9[IMPORTANT_COLS+other_cols].sort_values(['clean_last_name', 'application_year'])

match11 = match10.dropna(subset=[PERSON_ID], axis=0).sort_values(['clean_last_name', 'clean_first_name'])

# wide_apps5.to_pickle(os.path.join(APP_DATA_DIR, 'all_apps_plus_NIH_info.p'))
match11.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info.csv'), index=False)

wide_apps_v = match11.loc[(match11.application_year>1960) & (match11.application_year<1976), :].sort_values(
    ['clean_last_name', 'application_date'])

wide_apps_v.to_pickle(os.path.join(PICKLE_DIR, 'fuzzy_all_apps_plus_NIH_info_vietnam.p'))

wide_apps_v.to_csv(os.path.join(APP_DATA_DIR, 'fuzzy_all_apps_plus_NIH_info_vietnam.csv'), index=False)

In [ ]:
ln = 'RING'
wide_apps_v.loc[wide_apps_v.clean_last_name==ln, NAME_COLS+[PERSON_ID, 'medical_school', 'application_year', 'dno', 'residency','residency_hospital',
                                                            'internship_hospital', 'residency_dates']]

In [ ]:
wide_apps_v.loc[wide_apps_v.clean_last_name=='KRUGMAN', NAME_COLS+['dno', PERSON_ID]]

In [ ]:
wide_apps_v.loc[pd.isnull(wide_apps_v.clean_first_name), NAME_COLS+['dno', 'medical_school', 'application_year']].to_csv('extras.csv')

In [ ]:
apps5.loc[apps5.clean_last_name=='PRZYBYLA', NAME_COLS+['medical_school', PERSON_ID]]